In [1]:
words = open("names.txt", 'r').read().splitlines()
charsInNames = sorted(list(set(''.join(words))))

In [2]:
import torch
import torch.nn.functional as F

In [3]:
#Tokenization
# Initialize stoi with '.' mapped to 0
stoi = {'.': 0}

# Add mappings for characters in charsInNames, starting from 1
stoi.update({ch: i for i, ch in enumerate(charsInNames, 1)})

itos = {i:ch for ch,i in stoi.items()}

In [4]:
# Create a tensor of zeros with dimensions (27, 27)
x ,y = [], []

# for w in words[:1]: #test for first name
for w in words:
  word = ['.'] + list(w) + ['.'] # we want to save the occurance of the char at the end and begining
  for ch1, ch2 in zip(word, word[1:]): #the rows are first char and the columns are the second char
    ch1tot = stoi[ch1]
    ch2tot = stoi[ch2]
    x.append(ch1tot)
    y.append(ch2tot)
    
x = torch.tensor(x)
y = torch.tensor(y)


In [11]:
num = x.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

number of examples:  228146


In [ ]:
# lop for loss

for k in range(500):
  
  # forward pass
  x_one_hot = F.one_hot(x, num_classes=27).float()
  logits = x_one_hot @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(num), y].log().mean() + 0.01*(W**2).mean()
  
  
  # backward propegation
  W.grad = None
  loss.backward()
  W.data += -50 * W.grad


print(loss.item())

In [ ]:

g = torch.Generator().manual_seed(2147483647)
for i in range(5):
  
  out = []
  ix = 0
  while True:

    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W 
    counts = logits.exp() 
    p = counts / counts.sum(1, keepdims=True) 

    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))